**LLM Workshop 2024 by Sebastian Raschka**

This code is based on *Build a Large Language Model (From Scratch)*, [https://github.com/rasbt/LLMs-from-scratch](https://github.com/rasbt/LLMs-from-scratch)

# Setup

In [ ]:
# Requirements from: https://github.com/rasbt/LLM-workshop-2024/blob/main/requirements.txt
requirements = """
torch >= 2.0.1
tiktoken >= 0.5.1
matplotlib >= 3.7.1
numpy >= 1.24.3
tensorflow >= 2.15.0
tqdm >= 4.66.1
numpy >= 1.25, < 2.0
pandas >= 2.2.1
psutil >= 5.9.5
litgpt[all] >= 0.4.1
"""

with open("requirements.txt", mode="wt") as f:
    f.write(requirements)

%pip install -r requirements.txt --quiet

# 2) Understanding LLM Input Data

Packages that are being used in this notebook:

In [1]:
from importlib.metadata import version


print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.2.1+cu121
tiktoken version: 0.7.0


- This notebook provides a brief overview of the data preparation and sampling procedures to get input data "ready" for an LLM
- Understanding what the input data looks like is a great first step towards understanding how LLMs work

<img src="https://github.com/rasbt/LLM-workshop-2024/blob/main/02_data/figures/01.png?raw=1" width="1000px">

<br>
<br>
<br>
<br>

# 2.1 Tokenizing text

- In this section, we tokenize text, which means breaking text into smaller units, such as individual words and punctuation characters

<img src="https://github.com/rasbt/LLM-workshop-2024/blob/main/02_data/figures/02.png?raw=1" width="800px">

- Load raw text we want to work with
- [The Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict) is a public domain short story

In [2]:
import requests

session = requests.Session()
with open("the-verdict.txt", "wt", encoding="utf-8") as f:
    response = session.get(
        "https://raw.githubusercontent.com/rasbt/LLM-workshop-2024/main/02_data/the-verdict.txt"
    )
    f.write(response.text)

In [3]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print(f"Total number of character: {len(raw_text):,d}")
print(raw_text[:99])

Total number of character: 20,479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- The goal is to tokenize and embed this text for an LLM
- Let's develop a simple tokenizer based on some simple sample text that we can then later apply to the text above

<img src="https://github.com/rasbt/LLM-workshop-2024/blob/main/02_data/figures/03.png?raw=1" width="690px">

- The following regular expression will split on whitespaces and punctuation

In [7]:
import re

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
# preprocessed = [item for item in preprocessed if item]
print(preprocessed[:38])

['I', ' ', 'HAD', ' ', 'always', ' ', 'thought', ' ', 'Jack', ' ', 'Gisburn', ' ', 'rather', ' ', 'a', ' ', 'cheap', ' ', 'genius', '--', 'though', ' ', 'a', ' ', 'good', ' ', 'fellow', ' ', 'enough', '--', 'so', ' ', 'it', ' ', 'was', ' ', 'no', ' ']


In [5]:
print(f"Number of tokens: {len(preprocessed):,d}")

Number of tokens: 8,405


<br>
<br>
<br>
<br>

# 2.2 Converting tokens into token IDs

- Next, we convert the text tokens into token IDs that we can process via embedding layers later
- For this we first need to build a vocabulary

<img src="https://github.com/rasbt/LLM-workshop-2024/blob/main/02_data/figures/04.png?raw=1" width="900px">

- The vocabulary contains the unique words in the input text

In [6]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(f"{vocab_size:,d} tokens")

1,132 tokens


In [8]:
# Here are the first few tokens
print(all_words[:20])

['\n', ' ', '!', '"', "'", '(', ')', ',', '--', '.', ':', ';', '?', 'A', 'Ah', 'Among', 'And', 'Are', 'Arrt', 'As']


In [9]:
vocab = {token: integer for integer, token in enumerate(all_words)}

- Below are the first 50 entries in this vocabulary:

In [11]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('\n', 0)
(' ', 1)
('!', 2)
('"', 3)
("'", 4)
('(', 5)
(')', 6)
(',', 7)
('--', 8)
('.', 9)
(':', 10)
(';', 11)
('?', 12)
('A', 13)
('Ah', 14)
('Among', 15)
('And', 16)
('Are', 17)
('Arrt', 18)
('As', 19)
('At', 20)
('Be', 21)
('Begin', 22)
('Burlington', 23)
('But', 24)
('By', 25)
('Carlo', 26)
('Chicago', 27)
('Claude', 28)
('Come', 29)
('Croft', 30)
('Destroyed', 31)
('Devonshire', 32)
('Don', 33)
('Dubarry', 34)
('Emperors', 35)
('Florence', 36)
('For', 37)
('Gallery', 38)
('Gideon', 39)
('Gisburn', 40)
('Gisburns', 41)
('Grafton', 42)
('Greek', 43)
('Grindle', 44)
('Grindles', 45)
('HAD', 46)
('Had', 47)
('Hang', 48)
('Has', 49)
('He', 50)


- Below, we illustrate the tokenization of a short sample text using a small vocabulary:

In [12]:
# Quick test on converting the tokens into integer representations
new_text = re.split(r'([,.:;?_!"()\']|--|\s)', "This is a new sentence")
[vocab.get(token, "[UNK]") for token in new_text]

[99, 1, 586, 1, 117, 1, '[UNK]', 1, '[UNK]']

<img src="https://github.com/rasbt/LLM-workshop-2024/blob/main/02_data/figures/05.png?raw=1" width="800px">

- Let's now put it all together into a tokenizer class

In [13]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        # Include a token for tokens not in the training dataset vocabulary
        vocab["[UNK]"] = len(vocab)
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}
        self.unknown_token_int = self.str_to_int["[UNK]"]  # same as len(vocab)

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int.get(s, self.unknown_token_int) for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r"\1", text)
        return text

- The `encode` function turns text into token IDs
- The `decode` function turns token IDs back into text

<img src="https://github.com/rasbt/LLM-workshop-2024/blob/main/02_data/figures/06.png?raw=1" width="800px">

- We can use the tokenizer to encode (that is, tokenize) texts into integers
- These integers can then be embedded (later) as input of/for the LLM

In [14]:
tokenizer = SimpleTokenizerV1(vocab)

text = """
    "It's the last he painted, you know,"
    Mrs. Gisburn said with pardonable pride.
    """
ids = tokenizer.encode(text)
print(ids)
print(tokenizer.decode(ids))

[3, 58, 4, 852, 990, 604, 535, 748, 7, 1128, 598, 7, 3, 69, 9, 40, 853, 1110, 756, 795, 9]
" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


- We can decode the integers back into text

In [ ]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [ ]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [15]:
tokenizer.decode(tokenizer.encode("This is a new sentence"))

'This is a [UNK] [UNK]'

---

# 2.3 BytePair encoding

- GPT-2 used BytePair encoding (BPE) as its tokenizer
- it allows the model to break down words that aren't in its predefined vocabulary into smaller subword units or even individual characters, enabling it to handle out-of-vocabulary words
- For instance, if GPT-2's vocabulary doesn't have the word "unfamiliarword," it might tokenize it as ["unfam", "iliar", "word"] or some other subword breakdown, depending on its trained BPE merges
- The original BPE tokenizer can be found here: [https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- In this lecture, we are using the BPE tokenizer from OpenAI's open-source [tiktoken](https://github.com/openai/tiktoken) library, which implements its core algorithms in Rust to improve computational performance
- (Based on an analysis [here](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/02_bonus_bytepair-encoder/compare-bpe-tiktoken.ipynb), I found that `tiktoken` is approx. 3x faster than the original tokenizer and 6x faster than an equivalent tokenizer in Hugging Face)

In [ ]:
# pip install tiktoken

In [16]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.7.0


In [17]:
tokenizer = tiktoken.get_encoding("gpt2")

In [18]:
text = (
    "Hello, do you like watermelon? <|endoftext|> In the sunlit terraces"
    " of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 1660, 45690, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]


In [19]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like watermelon? <|endoftext|> In the sunlit terraces of someunknownPlace.


- BPE tokenizers break down unknown words into subwords and individual characters:

<img src="https://github.com/rasbt/LLM-workshop-2024/blob/main/02_data/figures/07.png?raw=1" width="700px">

In [20]:
for i in tokenizer.encode("Akwirw ier", allowed_special={"<|endoftext|>"}):
    print(str(i).ljust(10, "."), tokenizer.decode([i]), sep="")

33901.....Ak
86........w
343.......ir
86........w
220....... 
959.......ier


In [21]:
for i in tokenizer.encode(
    "The Hubble Deep Field image peers into the furthest reaches of the universe...",
    allowed_special={"<|endoftext|>"},
):
    print(str(i).ljust(10, "_"), tokenizer.decode([i]), sep="")

464_______The
40455_____ Hubble
10766_____ Deep
7663______ Field
2939______ image
14495_____ peers
656_______ into
262_______ the
46186_____ furthe
301_______st
12229_____ reaches
286_______ of
262_______ the
6881______ universe
986_______...


<br>
<br>
<br>
<br>

# 2.4 Data sampling with a sliding window

- Above, we took care of the tokenization (converting text into word tokens represented as token ID numbers)
- Now, let's talk about how we create the data loading for LLMs
- We train LLMs to generate one word at a time, so we want to prepare the training data accordingly where the next word in a sequence represents the target to predict

<img src="https://github.com/rasbt/LLM-workshop-2024/blob/main/02_data/figures/08.png?raw=1" width="800px">

- For this, we use a sliding window approach, changing the position by +1:

<img src="https://github.com/rasbt/LLM-workshop-2024/blob/main/02_data/figures/09.png?raw=1" width="900px">

- Note that in practice it's best to set the stride equal to the context length so that we don't have overlaps between the inputs (the targets are still shifted by +1 always)

<img src="https://github.com/rasbt/LLM-workshop-2024/blob/main/02_data/figures/10.png?raw=1" width="800px">

In [22]:
session = requests.Session()
with open("custom_dataloader.py", "wt", encoding="utf-8") as f:
    response = session.get(
        "https://raw.githubusercontent.com/rasbt/LLM-workshop-2024/main/02_data/supplementary.py"
    )
    f.write(response.text)

In [23]:
from custom_dataloader import create_dataloader_v1


dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=4, stride=4, shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


<br>
<br>
<br>
<br>

# Exercise: Prepare your own favorite text dataset

Dataset: Jane Austen's books, from Project Gutenberg

In [24]:
import re  # regular expressions, for extracting text from books
import requests  # for accessing books from URLs
from pathlib import Path  # for creating a directory to store books

def get_jane_austen_book_text():
    """Return the full text of Jane Austen's books on Project Gutenberg"""
    book_names = [
        "Pride and Prejudice",
        "Emma",
        "Sense and Sensibility",
        "Persuasion",
        "Northanger Abbey",
        "Mansfield Park",
        "Lady Susan",
    ]

    book_urls = [
        "https://www.gutenberg.org/files/1342/1342-0.txt",
        "https://www.gutenberg.org/cache/epub/158/pg158.txt",
        "https://www.gutenberg.org/files/161/161-0.txt",
        "https://www.gutenberg.org/cache/epub/105/pg105.txt",
        "https://www.gutenberg.org/files/121/121-0.txt",
        "https://www.gutenberg.org/files/141/141-0.txt",
        "https://www.gutenberg.org/cache/epub/946/pg946.txt",
    ]

    # Create a folder to save the books in
    book_directory = Path("./books/")
    if not book_directory.exists():
        book_directory.mkdir()

    jane_austen_full_text = ""

    session = requests.Session()

    # Save books to a folder so they can be accessed later if needed
    # Also save the entire text in the jane_austen_full_text variable
    for num, url in enumerate(book_urls):
        response = session.get(url)
        # Ensure the response text is properly formatted as utf-8
        # See: https://stackoverflow.com/a/44203507/17005348
        response.encoding = "UTF-8"
        book_text = response.text
        book_title = book_names[num].replace(" ", "_")
        with open(f"./books/{book_title}.txt", mode="w+", encoding="utf-8") as book_file:
            book_file.write(book_text)

        # Save the text to the jane_austen_full_text variable,
        # removing the intro and end parts from Project Gutenberg
        # which have this form: *** BEGIN BOOK *** or *** END OF BOOK ***
        jane_austen_full_text += re.split(r"[*]{3}.*?[*]{3}", string=book_text)[1]

    with open("./books/jane_austen.txt", mode="w+", encoding="utf-8") as book_file:
        book_file.write(jane_austen_full_text)
    
    print("Full text saved to: ./books/jane_austen.txt")
    print(f"Total number of characters: {len(jane_austen_full_text):,d}")
    return jane_austen_full_text

jane_austen_full_text = get_jane_austen_book_text()

Total number of characters: 4,268,236


In [25]:
dataloader = create_dataloader_v1(
    jane_austen_full_text, batch_size=8, max_length=8, stride=8, shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[  201,   198,   201,   198,   201,   198,   201,   198],
        [  201,   198,   220,   220,   220,   220,   220,   220],
        [  220,   220,   220,   220,   220,   220,   220,   220],
        [  220,   220,   220,   220,   220,   220,   220,   220],
        [  220,   220,   220,   220,   220,   685, 21478, 44027],
        [   25,   201,   198,   201,   198,   220,   220,   220],
        [  220,   220,   220,   220,   220,   220,   220,   220],
        [  220,   220,   220,   220,   220,   220,   220,   220]])

Targets:
 tensor([[  198,   201,   198,   201,   198,   201,   198,   201],
        [  198,   220,   220,   220,   220,   220,   220,   220],
        [  220,   220,   220,   220,   220,   220,   220,   220],
        [  220,   220,   220,   220,   220,   220,   220,   220],
        [  220,   220,   220,   220,   685, 21478, 44027,    25],
        [  201,   198,   201,   198,   220,   220,   220,   220],
        [  220,   220,   220,   220,   220,   220,   22